<a href="https://colab.research.google.com/github/jasonmshaw05/python-learning-journal/blob/main/Week5%266Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
#linked list implementation
class Node:
  def __init__(self, data):
    self.data = data
    self.next = None

class LinkedList:
  def __init__(self):
    self.size = 0
    self.head = None

  def prepend(self,data):
    newNode = Node(data)
    newNode.next = self.head
    self.head = newNode
    self.size += 1

  def append(self,data):
    newNode = Node(data)
    newNode.next = None
    if self.head == None:
      self.head = newNode
    else:
      nodeAt = self.head
      while nodeAt.next != None:
        nodeAt = nodeAt.next
      nodeAt.next = newNode
    self.size += 1

  def printList(self):
    nodeToPrint = self.head
    while nodeToPrint.next != None:
      print(nodeToPrint.data)
      nodeToPrint = nodeToPrint.next
    print(nodeToPrint.data)

  def search(self, target):
    nodeToSearch = self.head
    while nodeToSearch.next != None:
      if nodeToSearch.data == target:
        return nodeToSearch
      nodeToPrint = nodeToPrint.next
    return None

  def insertAfter(self, index, data):
    newNode = Node(data)
    nodeAt = self.head
    nodeNext = nodeAt.next
    for x in range(index):
      nodeAt = nodeAt.next
      nodeNext = nodeNext.next
    nodeAt.next = newNode
    newNode.next = nodeNext
    self.size += 1

  def remove(self,position):
    nodeAt = self.head
    if position == 1:
      self.head = nodeAt.next
    elif position == self.size:
      for x in range(position-2):
        nodeAt = nodeAt.next
      nodeAt.next = None
    else:
      for x in range(position-2):
        nodeAt = nodeAt.next
      nodeAt.next = nodeAt.next.next





In [56]:
#classes UnitCard and SpellCard that inherit from a parent Card class
class Card:
  def __init__(self, name, cost):
    self.name = None
    self.cost = None

class UnitCard(Card):
  def __init__(self, name, cost, attack, health, boardIcon):
    self.cardType = "Unit"
    self.name = name
    self.cost = cost
    self.health = health
    self.attack = attack
    self.boardIcon = boardIcon

class SpellCard(Card):
  def __init__(self, name, cost):
    self.cardType = "Spell"
    self.name = name
    self.cost = cost

In [84]:
#player class that stores their "avatar", health, max and temp mana. Generates a random deck on creation, an empty hand, and a board filled with null minions
class Player:
  def __init__(self, ID):
    self.ID = ID
    self.health = 30
    self.maxMana = 5 #if mana is a fraction (like in hearthstone), max mana is the total amount you get per turn [0/X]
    self.tempMana = 0 #temp mana is the current amount you have [X/0]
    self.deck = createDeck(30)
    self.hand = []
    self.board = [UnitCard("EmptyPos",0,0,0,"0")] * 5

#draws cards for the player of the amount given
  def drawCards(self, amount):
    for cards in range(amount):
      self.hand.append(self.deck.head.data)
      self.deck.remove(1) #removes the card from the top of the deck
    print("Player " + self.ID + " drew " + str(amount) + " cards!")

#returns the hand size for checking to see if a valid card was picked in getPlayerChoice()
  def getHandSize(self):
    return len(self.hand)

In [58]:
def createDeck(cards):
  #randomly generate a deck of 30 cards, 15 unit and 15 spell, per player
  playerDeck = LinkedList()
  unitCount = 0
  spellCount = 0
  #to ensure a random deck, either a unit or a spell is randomly generated, unless there are more than 10 of either, in which case only the latter is generated
  for x in range (0,cards):
    if random.randint(0,1) == 0 and unitCount < 15:
      playerDeck.append(generateUnitCard())
      unitCount += 1
    elif spellCount < 15:
      playerDeck.append(generateSpellCard())
      spellCount += 1
    else:
      playerDeck.append(generateUnitCard())
      unitCount += 1
  return playerDeck

In [88]:
def generateUnitCard():
  unitChoice = random.randint(0,10)
  if unitChoice <=  3:
    return UnitCard("Gnome Archer", 1, 2, 3, "G") #2/3 that costs 1 mana (power creep amirite?)
  elif unitChoice <= 6:
    return UnitCard("Back-alley Thug", 2, 4, 2, "B") #4/2 that costs 2 mana
  elif unitChoice <= 8:
    return UnitCard("Orc Shieldmistress", 3, 3, 8, "S") #2/9 that costs 3 mana
  elif unitChoice == 9:
    return UnitCard("Kenku Wanderer", 4, 7, 5, "K") #8/6 that costs 4 mana
  elif unitChoice == 10:
    return UnitCard("Elemental Wyrm", 6, 11, 11, "E") #11/11 that costs 6 mana

In [89]:
def generateSpellCard():
  spellChoice = random.randint(0,9)
  if spellChoice <= 2:
    return SpellCard("Fire Spire", 1) #deal 3 damage to a MINION only
  elif spellChoice <= 5:
    return SpellCard("Vitality Boon", 1) #buff a minion's health by 5
  elif spellChoice <= 7:
    return SpellCard("Earthquake Rune", 2) #deal 2 damage to all enemy characters
  if spellChoice == 8:
    return SpellCard("Switcheroo", 2) #swap two units on the board
  elif spellChoice == 9:
    return SpellCard("Second Wind", 3) #heal 12 health to your avatar

In [86]:
from google.colab import output
import time
import random

def runGame():
  player1 = Player("1_1") #creates two players
  player2 = Player("2_2")
  turn = 0
  player1.drawCards(6) #they each draw 6 cards
  player2.drawCards(6)
  leadingPlayer, laggingPlayer = coinFlip(player1, player2) #a first and second player (leading and lagging) are set using coinFlip()
  time.sleep(5)
  while leadingPlayer.health > 0 and laggingPlayer.health > 0: #for however long that both players are above 0 health, setup and battle are called for the leading player,
    turn += 1
    setupPhase(leadingPlayer, laggingPlayer, turn)
    battlePhase(leadingPlayer, laggingPlayer)
    if laggingPlayer.health <= 0: #if the leading player drops their opponent to 0 or below HP, the leading player wins
      print(leadingPlayer.ID + " Wins!")
      break
    leadingPlayer.drawCards(3)
    if (leadingPlayer.deck.size == 0):
      suddenDeath(leadingPlayer, laggingPlayer)
      break
    turn += 1
    setupPhase(laggingPlayer, leadingPlayer, turn) #the process is repeated for the lagging player
    battlePhase(laggingPlayer, leadingPlayer)
    if leadingPlayer.health <= 0:
      print(laggingPlayer.ID + " Wins!")
    laggingPlayer.drawCards(3)

In [62]:
#decide leading and lagging, print some explanation text, and iterate the lagging player's mana by 1, returning the order of the leading and lagging player
def coinFlip(player1, player2):
  coinFlip = random.randint(0,1)
  if coinFlip == 0:
    print("The coin landed on heads! Player 1 starts and Player 2 will receive 1 extra mana in exchange")
    print()
    player2.maxMana += 1
    return player1, player2
  else:
    print("The coin landed on tails! Player 2 starts and Player 1 will receive 1 extra mana in exchange")
    print()
    player1.maxMana += 1
    return player2, player1

In [78]:
#players place units and spells during this phase
def setupPhase(currentPlayer, offPlayer, turn):
  if currentPlayer.maxMana < 10: #mana is iterated by 1, unless mana is aleady at 10
    currentPlayer.maxMana += 1
  currentPlayer.tempMana = currentPlayer.maxMana
  endLoop = False
  while endLoop == False: #allows players to play cards until they desire to end their turn
    output.clear()
    print("Turn " + str(turn))
    generateBoard(currentPlayer, offPlayer)
    generateStats(currentPlayer, offPlayer)
    generateHand(currentPlayer, offPlayer)
    print("Which card would you like to play? (Type 'pass' to skip your turn)")
    playerChoice = getPlayerChoice(currentPlayer)
    if playerChoice == -1:
      endLoop = endTurn()
    else:
      currentPlayer.tempMana -= currentPlayer.hand[playerChoice-1].cost #the cost of the card is subtracted from the mana total
      if currentPlayer.hand[playerChoice-1].cardType == "Unit":
        unitImplementation(currentPlayer, offPlayer, playerChoice-1) #if the card is a unit, a position is chosen for the unit
      elif currentPlayer.hand[playerChoice-1].cardType == "Spell":
        spellImplementation(currentPlayer, offPlayer, playerChoice-1) #if the card is a spell, a target is chosen and the effect takes place
      endLoop = endTurn()

In [64]:
#returns the card choice
def getPlayerChoice(currentPlayer):
  playerInput = input()
  endLoop = False
  while endLoop == False:
      if playerInput.lower() == "pass":
        return -1
      try:
        if int(playerInput) > currentPlayer.getHandSize() or int(playerInput) < 1: #if the player inputs a number greater than the hand size or less than 1, an error is thrown
          print("Not a valid card")
          playerInput = input()
        else:
          if currentPlayer.hand[int(playerInput)-1].cost > currentPlayer.tempMana: #if they don't have enough mana to play the card, an error is thrown
            print("You don't have enough mana to play this card!")
            playerInput = input()
          else:
            return int(playerInput) #otherwise, the card choice is returned
            endLoop = True
      except:
        print("Not a valid input") #if the input is not a number, an error is thrown
        playerInput = input()

#returns either true or false if the player ends their turn
def endTurn():
  print("End turn? (Y/N)")
  endLoop = False
  playerInput = input()
  while endLoop == False:
    if playerInput == "Y":
      return True
    elif playerInput == "N":
      return False
    else:
      print("Invalid input")
      playerInput = input()

In [65]:
def unitImplementation(currentPlayer, offPlayer, playerChoice):
  print("What position would you like to play your unit?")
  playerInput = input()
  endLoop = False
  while endLoop == False:
    try:
      if int(playerInput) > 5 or int(playerInput) < 1:
        print("Not a valid position")
        playerInput = input()
      else:
        output.clear()
        currentPlayer.board[int(playerInput)-1] = currentPlayer.hand.pop(playerChoice) #the pop method is specifically performed to REMOVE the card from your hand
        endLoop = True
        generateBoard(currentPlayer,offPlayer)
        generateStats(currentPlayer, offPlayer)
        generateHand(currentPlayer,offPlayer)
    except:
      print("Invalid input")
      playerInput = input()

In [77]:
def spellImplementation(currentPlayer, offPlayer, playerChoice):
  spellChosen = currentPlayer.hand.pop(playerChoice)
  if spellChosen.name == "Fire Spire":
    target = spellSelector(currentPlayer, offPlayer)
    if target < 5:
      currentPlayer.board[target].health -= 3
      print("Your " + currentPlayer.board[target].name + " took 3 damage!")
      if currentPlayer.board[target-5].health <= 0:
        print("Your " + currentPlayer.board[target].name + " has been slain!")
        offPlayer.board[target] =  UnitCard("EmptyPos",0,0,0,"0")
    else:
      offPlayer.board[target-5].health -= 3
      print("Opponent's " + offPlayer.board[target-5].name + " took 3 damage!")
      if offPlayer.board[target-5].health <= 0:
        print("Opponent's " + offPlayer.board[target-5].name + " has been slain!")
        offPlayer.board[target-5] =  UnitCard("EmptyPos",0,0,0,"0")
  elif spellChosen.name == "Vitality Boon":
    target = spellSelector(currentPlayer, offPlayer)
    if target < 5:
      currentPlayer.board[target].health += 5
      print("Your " + currentPlayer.board[target].name + " received 5 extra HP!")
    else:
      offPlayer.board[target-5].health += 5
      print("Opponent's " + offPlayer.board[target-5].name + " received 5 extra HP!")
  elif spellChosen.name == "Earthquake Rune":
    print("All units on the enemy's board took 2 damage!")
    for units in range(5):
      offPlayer.board[units].health -= 2
      if offPlayer.board[units].health <= 0 and offPlayer.board[units].boardIcon != "0":
        print("Opponent's " + offPlayer.board[units].name + " has been slain!")
        offPlayer.board[units] =  UnitCard("EmptyPos",0,0,0,"0")
    offPlayer.health -= 2
  elif spellChosen.name =="Switcheroo":
    print("For your first monster,")
    target1 = spellSelector(currentPlayer, offPlayer)
    print("For your second monster,")
    target2 = spellSelector(currentPlayer, offPlayer)
    if target1 < 5:
      temp = currentPlayer.board[target1]
      if target2 < 5:
        print("Target 1:")
        print(currentPlayer.board[target1].name)
        print("Target 2:")
        print(currentPlayer.board[target2].name)
        currentPlayer.board[target1] = currentPlayer.board[target2]
        currentPlayer.board[target2] = temp
      else:
        target2 -= 5
        print("Target 1:")
        print(currentPlayer.board[target1].name)
        print("Target 2:")
        print(offPlayer.board[target2].name)
        currentPlayer.board[target1] = offPlayer.board[target2]
        offPlayer.board[target2] = temp
    else:
      target1 -= 5
      temp = offPlayer.board[target1]
      if target2 < 5:
        print("Target 1:")
        print(offPlayer.board[target1].name)
        print("Target 2:")
        print(currentPlayer.board[target2].name)
        offPlayer.board[target1] = currentPlayer.board[target2]
        currentPlayer.board[target2] = temp
      else:
        target2 -= 5
        print("Target 1:")
        print(offPlayer.board[target1].name)
        print("Target 2:")
        print(offPlayer.board[target2].name)
        offPlayer.board[target1] = offPlayer.board[target2]
        offPlayer.board[target2] = temp
    print("The two units swapped places!")
  elif spellChosen.name == "Second Wind":
    currentPlayer.health += 12
    print("You have been healed for 12 damage!")


In [75]:
def spellSelector(currentPlayer, offPlayer):
  print("Choose a target for your spell (1-5 for your units [left to right], 6-10 for your opponent's units)")
  playerInput = input()
  endLoop = False
  while endLoop == False:
    try:
      if int(playerInput) > 10 or int(playerInput) < 1:
        print("Not a valid position")
        playerInput = input()
      elif int(playerInput) <= 5:
        if currentPlayer.board[int(playerInput)-1].health == 0:
          print("There is no unit at that location")
          playerInput = input()
        else:
          return int(playerInput) - 1
      else:
        if offPlayer.board[int(playerInput)-6].health == 0:
          print("There is no unit at that location")
          playerInput = input()
        else:
          return int(playerInput) - 1
    except:
      print("Invalid input")
      playerInput = input()

In [82]:
#iterating through each column, units played will either attack the opposing unit or the opposing player. Unlike hearthstone, attacks only hurt the defender and not the attacker as well.
def battlePhase(currentPlayer, offPlayer):
  output.clear()
  print("---------------------")
  print("Battle phase!")
  time.sleep(1)
  for cards in range(5):
    print("Column " + str(cards+1) + " attacks!")
    if currentPlayer.board[cards].boardIcon == "0": #if there is no unit in play
      print("You have no unit to attack with in this column")
    else:
      if offPlayer.board[cards].health <= 0:
        print("Opponent has no unit to defend and takes " + str(currentPlayer.board[cards].attack) + " damage!") #if there is no unit to defend
        offPlayer.health -= currentPlayer.board[cards].attack
      else:
        print("Opponent's " + str(offPlayer.board[cards].name) + " takes " + str(currentPlayer.board[cards].attack) + " damage!") #if there is a unit to defend
        offPlayer.board[cards].health -= currentPlayer.board[cards].attack
        if offPlayer.board[cards].health <= 0:
          print("Opponent's " + offPlayer.board[cards].name + " has been slain!") #if the defending unit dies
          offPlayer.board[cards] =  UnitCard("EmptyPos",0,0,0,"0")
    time.sleep(1)
    print()
  generateBoard(currentPlayer, offPlayer)
  generateStats(currentPlayer, offPlayer)
  print("Input anything to continue") #acts as a buffer for the player to decide on their next move or just take in the board state
  input()

In [69]:
def suddenDeath(leadingPlayer, laggingPlayer):
  print("Player " + leadingPlayer.ID + " is out of cards! Because of this, the game has entered sudden death! The person with the least HP loses!")
  print("Player " + leadingPlayer.ID + ": " + str(leadingPlayer.health) + " health")
  print("Player " + laggingPlayer.ID + ": " + str(laggingPlayer.health) + " health")
  if leadingPlayer.health > laggingPlayer.health:
    print("Player " + leadingPlayer.ID + " wins!")
  elif leadingPlayer.health < laggingPlayer.health:
    print("Player " + laggingPlayer.ID + " wins!")
  else:
    print("Both players have the same HP! The game ends in a tie")

In [70]:
#prints a 2D array of each player's mana, health, avatars, and unit icons on the board.
def generateBoard(currentPlayer, offPlayer):
  print("Current game board:")
  print("---------------------")
  TwoD = [["MP:" + str(offPlayer.tempMana) + "  " + str(offPlayer.ID) + "  HP:" + str(offPlayer.health)],
          [" " + "_" + " " + "_" + " " + "_" + " " + "_" + " " + "_" + " " + "_" + " "],
          ["|" + " " + offPlayer.board[0].boardIcon + " " + offPlayer.board[1].boardIcon + " " + offPlayer.board[2].boardIcon + " " + offPlayer.board[3].boardIcon + " " + offPlayer.board[4].boardIcon + " " + "|"],
          [" " + "-" + " " + "-" + " " + "-" + " " + "-" + " " + "-" + " " + "-" + " "],
          ["|" + " " + currentPlayer.board[0].boardIcon + " " + currentPlayer.board[1].boardIcon + " " + currentPlayer.board[2].boardIcon + " " + currentPlayer.board[3].boardIcon + " " + currentPlayer.board[4].boardIcon + " " + "|"],
          [" " + "‾" + " " + "‾" + " " + "‾" + " " + "‾" + " " + "‾" + " " + "‾" + " "],
          ["MP:" + str(currentPlayer.tempMana) + "  " + str(currentPlayer.ID) + "  HP:" + str(currentPlayer.health)]]
  for arrays in TwoD:
    for elements in arrays:
      print(elements)
  print("---------------------")

In [74]:
def generateStats(currentPlayer, offPlayer):
  print("Your units: ")
  for x in range (1,6):
    if currentPlayer.board[x-1].name == "EmptyPos":
      print(str(x) + ". No unit")
    else:
      print(str(x) + ". " + currentPlayer.board[x-1].name + " (" + str(currentPlayer.board[x-1].attack) + "/" + str(currentPlayer.board[x-1].health) + ")")
  print()
  print("Opponent's units: ")
  for y in range (1,6):
    if offPlayer.board[y-1].name == "EmptyPos":
      print(str(x) + ". No unit")
    else:
      print(str(y) + ". " + offPlayer.board[y-1].name + " (" + str(offPlayer.board[y-1].attack) + "/" + str(offPlayer.board[y-1].health) + ")")
  print("---------------------")

In [72]:
def generateHand(currentPlayer, offPlayer):
  print("Current hand:")
  position = 0
  for cards in currentPlayer.hand:
    position += 1
    print(str(position) + ":" + cards.name)
  print()

In [ ]:
runGame()